### Speed comparison of gradient boosting libraries for shap values calculations

Here we compare CatBoost, LightGBM and XGBoost for shap values calculations on GPU.

We used Titan X Pascal for training and evaluation.

We use epsilon_normalized dataset from [here](https://www.csie.ntu.edu.tw/~cjlin/libsvmtools/datasets/binary/).

In [1]:
import os
import numpy as np
import scipy
import pandas as pd
import copy
import tqdm
import datetime
from sklearn import datasets
import catboost
import xgboost as xgb
import lightgbm as lgb
import time

In [2]:
catboost.__version__, lgb.__version__, xgb.__version__

('0.11.1', '2.2.3', '0.81')

In [ ]:
train_data, train_target = datasets.load_svmlight_file("epsilon_normalized")
test_data, test_target = datasets.load_svmlight_file("epsilon_normalized.t",)

### parameters

In [4]:
num_iters = 1000
lr = 0.1
max_bin = 128
gpu_device = '0' # specify your GPU (used for training)
random_state = 0

In [5]:
train_target[train_target == -1] = 0
test_target[test_target == -1] = 0

In [7]:
def preprocess_data(data, label=None, mode='train', boosting=None):
    assert boosting is not None
    
    if isinstance(data, scipy.sparse.csr_matrix):
        data = data.todense().A
    
    if boosting == 'xgboost':
        return xgb.DMatrix(data, label)
    elif boosting == 'lightgbm':
        if mode == 'train':
            return lgb.Dataset(data, label)
        else:
            return data
    elif boosting == 'catboost':
        return catboost.Pool(data, label)
    else:
        raise RuntimeError("Unknown boosting library")

In [8]:
def create_parameters(base_params, boosting=None, **kwargs):
    assert boosting is not None
    assert isinstance(base_params, dict)
    
    params = copy.copy(base_params)
    if boosting == 'xgboost':
        params['objective'] = 'binary:logistic'
        params['max_depth'] = kwargs['depth']
        params['tree_method'] = 'gpu_hist'
        params['gpu_id'] = gpu_device
    elif boosting == 'lightgbm':
        params['objective'] = 'binary'
        params['device'] = "gpu"
        params['gpu_device_id'] = gpu_device
        params['num_leaves'] = 2**kwargs['depth']
    elif boosting == 'catboost':
        params['objective'] = 'Logloss'
        params['task_type'] = 'GPU'
        params['devices'] = gpu_device
        params['bootstrap_type'] = 'Bernoulli'
        params['logging_level'] = 'Silent'
    else:
        raise RuntimeError("Unknown boosting library")
        
    return params

In [9]:
def train(data, params, num_iters, boosting=None):
    assert boosting is not None
    if boosting == 'xgboost':
        return xgb.train(params=params, dtrain=data, num_boost_round=num_iters)
    elif boosting == 'lightgbm':
        return lgb.train(params=params, train_set=data, num_boost_round=num_iters)
    elif boosting == 'catboost':
        return catboost.train(pool=data, params=params, num_boost_round=num_iters)
    else:
        raise RuntimeError("Unknown boosting library")

In [10]:
def predict_shap(model, data, boosting=None):
    assert boosting is not None
    if boosting == 'xgboost':
        return model.predict(data, pred_contribs=True)
    elif boosting == 'lightgbm':
        return model.predict(data, pred_contrib=True)
    elif boosting == 'catboost':
        return model.get_feature_importance(data, fstr_type='ShapValues')

In [11]:
def create_path(boosting, params):
    fname = [boosting]
    for key, value in sorted(params.items()):
        fname.append(str(key))
        fname.append(str(value))
    fname = "_".join(fname)
    fname = fname.replace(".", '')
    fname += ".model"
    return fname

In [12]:
def load_model(fname, boosting):
    if boosting == "xgboost":
        bst = xgb.Booster(model_file=fname)
        bst.load_model(fname)
    elif boosting == "lightgbm":
        bst = lgb.Booster(model_file=fname)
    elif boosting == "catboost":
        bst = catboost.CatBoost()
        bst.load_model(fname)
    else:
        raise RuntimeError("Unknown boosting")
    return bst

In [13]:
base_params = {
    'learning_rate': lr,
    'max_bin': max_bin,
    'random_state': random_state
}

In [15]:
lens_list = [x * 1000 for x in range(1, 11)]
lens_list

[1000, 2000, 3000, 4000, 5000, 6000, 7000, 8000, 9000, 10000]

In [ ]:
result = []

boosting_list = ['xgboost', 'catboost', 'lightgbm']
depth_list = [2, 4, 6, 8, 10]
lens_list = [x * 1000 for x in range(1, 11)]


for gb_type in boosting_list:
    
    print("{} is going".format(gb_type))
    
    for size_test in lens_list:
        print("size test {}".format(size_test))
        sep_test_data = test_data[:size_test]
        sep_test_target = test_target[:size_test]
        
        # comment this line if you have already trained all models
        train_preprocessed = preprocess_data(train_data, train_target, boosting=gb_type)
        
        test_preprocessed = preprocess_data(sep_test_data,
                                            sep_test_target, 
                                            mode='test',
                                            boosting=gb_type)

        for depth in tqdm.tqdm(depth_list):

            params = create_parameters(base_params, boosting=gb_type, depth=depth)
            params['depth'] = depth
            fname = create_path(gb_type, params)
            if os.path.exists(fname):
                print("model exist")
                bst = load_model(fname, boosting=gb_type)
            else:
                print("model is training")
                start_train = datetime.datetime.now()
                bst = train(train_preprocessed, params, num_iters=num_iters, boosting=gb_type)
                finish_train = datetime.datetime.now()
                delta_train = finish_train - start_train
                delta_train = int(delta_train.total_seconds() * 1000)
                bst.save_model(fname)

            start_time = datetime.datetime.now()
            preds = predict_shap(bst, test_preprocessed, boosting=gb_type)
            assert preds.shape == (sep_test_data.shape[0], sep_test_data.shape[1] + 1)
            finish_time = datetime.datetime.now()

            delta = finish_time - start_time
            delta = int(delta.total_seconds() * 1000)

            current_res = {
            'boosting': gb_type,
            'test_size': size_test,
            'depth': depth,
            'time': delta,
            }

            result.append(current_res)

        print("*" * 40)

In [25]:
result_df = pd.DataFrame(result)
result_df.head(2)

,boosting,depth,test_size,time
0,xgboost,2,1000,90
1,xgboost,4,1000,326


In [26]:
result_df.to_csv("shap_benchmark_{}_max_bin_with_test_sizes.csv".format(max_bin), index=False)

In [48]:
result_df = pd.read_csv("shap_benchmark_128_max_bin_with_test_sizes.csv", )
result_df['time'] = result_df['time'] / 1000.
result_df.pivot_table(index=["test_size", "depth"], columns="boosting", values="time")

boosting         catboost  lightgbm  xgboost
test_size depth                             
1000      2         0.463     0.087    0.090
          4         0.334     0.708    0.326
          6         0.812     6.092    2.148
          8         8.063    35.645   13.524
          10      136.618   183.836   52.084
2000      2         0.503     0.203    0.167
          4         0.428     1.595    0.654
          6         0.730    14.733    4.537
          8         5.330    74.180   26.107
          10      160.930   390.400  104.055
3000      2         0.841     0.229    0.227
          4         0.759     1.812    1.031
          6         1.168    18.385    6.593
          8         8.474   109.726   37.570
          10      161.769   542.842  150.296
4000      2         1.132     0.247    0.278
          4         1.065     2.619    1.295
          6         1.374    24.909    8.721
          8         8.946   143.655   48.192
          10      163.781   721.303  199.101
5000      2         1.120     0.324    0.324
          4         1.114     3.338    1.622
          6         1.661    31.314   11.520
          8         8.954   194.167   64.876
          10      162.450   796.446  241.245
6000      2         1.408     0.464    0.427
          4         1.310     4.026    2.005
          6         1.828    40.143   13.620
          8         9.226   209.503   77.800
          10      162.596  1076.097  312.067
7000      2         1.599     0.540    0.432
          4         1.692     4.662    2.132
          6         2.211    43.049   15.554
          8         9.273   252.127   94.719
          10      160.629  1164.899  356.502
8000      2         1.923     0.502    0.465
          4         1.888     5.326    2.412
          6         2.454    53.144   17.023
          8         9.953   288.688  101.261
          10      164.192  1355.683  388.989
9000      2         2.298     0.564    0.506
          4         2.348     5.605    2.674
          6         2.739    55.984   19.970
          8        10.154   373.078  117.172
          10      163.798  1619.727  486.676
10000     2         2.389     0.626    0.572
          4         2.492     6.327    3.056
          6         3.213    63.704   21.652
          8        10.292   328.536  129.460
          10      136.069  1726.045  491.275